<a href="https://colab.research.google.com/github/guebin/MP2024/blob/main/quiz/Quiz-11.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

| **항목**               | **허용 여부**        | **비고**                                          |
|------------------------|----------------------|---------------------------------------------------|
| **강의노트 참고**      | 허용                 | 수업 중 제공된 강의노트나 본인이 정리한 자료를 참고 가능       |
| **구글 검색**          | 허용                 | 인터넷을 통한 자료 검색 및 정보 확인 가능        |
| **생성 모형 사용**           | 허용            | 인공지능 기반 도구(GPT 등) 사용 가능            |

In [1]:
import os
os.environ["WANDB_MODE"] = "offline"

In [1]:
import pandas as pd
import transformers 
import datasets
import evaluate
import torch
import numpy as np

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. COVID19 tweets -- 100점

> 문항(1)-(2)를 모두 맞출 경우만 온전한 정답으로 인정. 즉 문항 (2)를 수행할 수 없는 구조로 (1)의 풀이를 설계할 경우 정답으로 인정하지 않음. 

>  평가지표를 올바르게 설정하지 않을 경우 30점 감점이 있음. 

>  시간절약을 위하여 일부의 데이터만 이용하여 학습한 경우 감점하지 않음. (코드만 맞게 쓰면..)

In [85]:
covid19_tweets_df = pd.read_csv('https://raw.githubusercontent.com/guebin/STML2022/main/posts/Corona_NLP_train.csv',encoding="ISO-8859-1")
covid19_tweets_df_train = covid19_tweets_df[::2].reset_index(drop=True)
covid19_tweets_df_test = covid19_tweets_df[1::2].reset_index(drop=True).loc[:,'UserName':'OriginalTweet'].assign(Sentiment = '???')
del covid19_tweets_df

In [86]:
covid19_tweets_df_train.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
2,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
3,3805,48757,"35.926541,-78.753267",16-03-2020,Cashier at grocery store was sharing his insig...,Positive
4,3807,48759,"Atlanta, GA USA",16-03-2020,Due to COVID-19 our retail store and classroom...,Positive


In [87]:
covid19_tweets_df_test.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,???
1,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,???
2,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,As news of the regionÂs first confirmed COVID...,???
3,3806,48758,Austria,16-03-2020,Was at the supermarket today. Didn't buy toile...,???
4,3808,48760,"BHAVNAGAR,GUJRAT",16-03-2020,"For corona prevention,we should stop to buy th...",???


이 데이터는 COVID-19 관련 트윗 데이터를 포함하며, 각 트윗에 대해 작성자의 감정을 나타내는 Sentiment(감정) 레이블이 지정되어 있다. 

**주요 열 설명**

`1`. OriginalTweet: 사용자가 작성한 트윗 내용으로. COVID-19와 관련된 다양한 의견, 경험, 상황 등이 포함되어 있음. 예시는 아래와 같음. 

- "Coronavirus Australia: Woolworths to give elders priority shopping amid coronavirus pandemic."
- “Due to COVID-19 our retail store and classroom...”

`2`. Sentiment: 트윗의 감정을 나타내는 레이블으로 가능한값은 아래와 같음. 

- Extremely Positive: 강한 긍정적 감정을 담아 희망과 연대감을 전달하는 트윗.
- Positive: 긍정적인 해결책이나 희망적인 메시지를 담고 있음.
- Neutral: 중립적이고 감정 표현이 없는 트윗 (사실 전달에 초점).
- Negative: 문제점이나 불만을 표현한 부정적인 트윗.
- Extremely Negative: 극단적으로 부정적인 상황에 대한 트윗.


`(1)` `transformers` 라이브러리의 "distilbert/distilbert-base-uncased" 모델을 활용하여 `OriginalTweet`을 입력으로 `Sentiment`를 예측하는 텍스트 분류기를 학습하라. 

**요구 사항**

`1`. 데이터 분할: 주어진 데이터셋 `covid19_tweets_df_train` 을 7:3 비율로 나누어, 70%는 훈련 데이터로, 나머지 30%는 검증 데이터로 사용하라. 

`2`. 평가지표: 모델의 성능평가를 위하여 아래의 함수를 사용하라. 

```Python
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)  # 예측 클래스

    # 평가 메트릭 불러오기
    acc = evaluate.load("accuracy")
    f1 = evaluate.load("f1")

    # 평가 메트릭 계산
    dct1 = acc.compute(predictions=predictions, references=labels)  
    dct2 = f1.compute(predictions=predictions, references=labels, average='weighted')

    # 두 개의 메트릭 결과를 합치기
    return dct1 | dct2  
```

`(2)` (1)에서 학습한 모델을 활용하여 `covid19_tweets_df_test`의 `OriginalTweet` 열에 대한 `Sentiment`를 예측하시오.

**답안예시**

In [77]:
covid19_tweets_df_test.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
1,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Extremely Positive
2,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,As news of the regionÂs first confirmed COVID...,Positive
3,3806,48758,Austria,16-03-2020,Was at the supermarket today. Didn't buy toile...,Neutral
4,3808,48760,"BHAVNAGAR,GUJRAT",16-03-2020,"For corona prevention,we should stop to buy th...",Negative


# 2. IMDB -- 가산점 50점

아래는 2주차에서 배운 `imdb` 분석코드를 정리한 것이다. 분석의 편의를 위하여 1000개의 자료만 사용하여 학습과 평가에 사용했다. 

In [2]:
## Step1 
imdb = datasets.load_dataset('imdb')
tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased") 
imdb_transformed = imdb.map(lambda dct: tokenizer(dct['text'],truncation=True), batched=True)
## Step2 
model = transformers.AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased", num_labels=2)
## Step3 
def accuracy(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    accuracy = evaluate.load("accuracy")
    return accuracy.compute(predictions=predictions, references=labels)
trainer = transformers.Trainer(
    model=model,
    args=transformers.TrainingArguments(
        output_dir="my_awesome_model",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=2,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        push_to_hub=False,
        report_to="none"
    ),
    train_dataset=imdb_transformed['train'].select(range(1000)), # 1000개만 사용
    eval_dataset=imdb_transformed['test'].select(range(1000)), # 1000개만 사용
    tokenizer=tokenizer,
    data_collator=transformers.DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=accuracy,
)
trainer.train()

Map: 100%|██████████████████████| 50000/50000 [00:04<00:00, 10520.76 examples/s]
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_116097/1501501835.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.002907,1.000000
2,No log,0.001795,1.000000


TrainOutput(global_step=126, training_loss=0.04093253801739405, metrics={'train_runtime': 31.5039, 'train_samples_per_second': 63.484, 'train_steps_per_second': 4.0, 'total_flos': 260867634212640.0, 'train_loss': 0.04093253801739405, 'epoch': 2.0})

보는것처럼 학습은 완벽하게 되었다. (accuracy가 100%) 그렇지만 `imdb_transformed['test']` 전체에서 평가를 해보니 결과가 좋지 않았다. (accuracy가 50%)

In [3]:
## Step4 
out = trainer.predict(imdb_transformed['test'])
out

PredictionOutput(predictions=array([[ 3.107105 , -3.2549791],
       [ 3.1408694, -3.259194 ],
       [ 3.0273504, -3.1274016],
       ...,
       [ 3.0847185, -3.2559047],
       [ 3.1443129, -3.2605338],
       [ 3.0649548, -3.2672653]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 1, 1]), metrics={'test_loss': 3.1562650203704834, 'test_accuracy': 0.5, 'test_runtime': 82.4169, 'test_samples_per_second': 303.336, 'test_steps_per_second': 18.965})

In [4]:
out.metrics['test_accuracy']

0.5

무엇이 문제일까? 혹시 아래의 코드를 수정하여 학습의 성능을 올릴 수 있는 방법이 있을까? (단 데이터의 수는 그대로 유지한다) 

```Python
trainer = transformers.Trainer(
    model=model,
    args=transformers.TrainingArguments(
        output_dir="my_awesome_model",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=2,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        push_to_hub=False,
        report_to="none"
    ),
    train_dataset=imdb_transformed['train'].select(range(1000)), # 1000개만 사용
    eval_dataset=imdb_transformed['test'].select(range(1000)), # 1000개만 사용
    tokenizer=tokenizer,
    data_collator=transformers.DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=accuracy,
)
```